# Optimize with LetTrade Grid Search

## Grid Search

### Sample Strategy

In [9]:
# import talib.abstract as ta

from lettrade import DataFeed, Strategy, indicator as i
from lettrade.exchange.backtest import ForexBackTestAccount, let_backtest
from lettrade.indicator.vendor.qtpylib import inject_indicators

inject_indicators()


class SmaCross(Strategy):
    ema1_period = 9
    ema2_period = 21

    def indicators(self, df: DataFeed):
        # df["ema1"] = ta.EMA(df, timeperiod=self.ema1_period)
        # df["ema2"] = ta.EMA(df, timeperiod=self.ema2_period)
        df["ema1"] = df.close.ema(window=self.ema1_period)
        df["ema2"] = df.close.ema(window=self.ema2_period)

        df["signal_ema_crossover"] = i.crossover(df.ema1, df.ema2)
        df["signal_ema_crossunder"] = i.crossunder(df.ema1, df.ema2)

    def next(self, df: DataFeed):
        if len(self.orders) > 0 or len(self.positionsonsonsons) > 0:
            return

        if df.l.signal_ema_crossover[-1]:
            price = df.l.close[-1]
            self.buy(size=0.1, sl=price - 0.001, tp=price + 0.001)
        elif df.l.signal_ema_crossunder[-1]:
            price = df.l.close[-1]
            self.sell(size=0.1, sl=price + 0.001, tp=price - 0.001)


lt = let_backtest(
    strategy=SmaCross,
    datas="example/data/data/EURUSD_5m_0_10000.csv",
    account=ForexBackTestAccount,
    # plotter=None,
)

### Optimize

`LetTrade` will auto cache optimize result

In [10]:
lt.optimize(ema1_period=[5, 6, 7, 8, 9, 10], ema2_period=range(10, 50, 1))

 41%|████▏     | 99/240 [00:00<00:00, 425.55it/s]

100%|██████████| 240/240 [00:01<00:00, 179.32it/s]


Rerun Optimize will reuse optimize result cached

In [11]:
lt.optimize(ema1_period=range(5, 25), ema2_period=range(10, 50, 1))

 14%|█▍        | 115/800 [00:00<00:02, 311.50it/s]

100%|██████████| 800/800 [00:01<00:00, 515.90it/s] 


### Plot

In [12]:
lt.plotter.heatmap(x="ema1_period", y="ema2_period", z="equity")

In [13]:
lt.plotter.contour(x="ema1_period", y="ema2_period", z="equity")

## Optimize from cache

Load optimize result from cache

In [14]:
lt.optimize_cache()

Load caches from: data/optimize
Loaded 943 caches


### Plot

In [15]:
lt.plotter.heatmap()

In [16]:
lt.plotter.contour()